In [9]:
def create_fitness_function(ann: ANN, X_train: np.ndarray, y_train: np.ndarray) -> Callable:
    """
    Create fitness function that evaluates ANN performance

    This function couples PSO to ANN:
    - Takes parameter vector from PSO particle
    - Sets ANN weights/biases using these parameters
    - Evaluates ANN on training data
    - Returns performance metric (MAE) as fitness

    Args:
        ann: The ANN architecture to evaluate
        X_train: Training input features (n_samples, n_features)
        y_train: Training target values (n_samples,)

    Returns:
        Fitness function that PSO will optimize
    """
    def fitness_function(parameter_vector: np.ndarray) -> float:
        """
        Evaluate ANN with given parameters

        Args:
            parameter_vector: Flat array of all ANN weights and biases

        Returns:
            Mean Absolute Error (MAE) on training data
            Lower values indicate better performance
        """
        # Decode PSO particle position into ANN parameters
        ann.set_parameters(parameter_vector)

        # Make predictions on training data
        predictions = ann.predict(X_train)

        # Calculate Mean Absolute Error
        mae = np.mean(np.abs(predictions.flatten() - y_train.flatten()))

        return mae

    return fitness_function

# ============================================================================
# DATA LOADING AND PREPROCESSING
# ============================================================================

def load_and_split_data(filepath: str, train_ratio: float = 0.7,
                        random_seed: int = 42) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Load concrete strength dataset and split into train/test sets

    Args:
        filepath: /content/concrete_data.csv
        train_ratio: Proportion of data for training (default 0.7 = 70%)
        random_seed: Random seed for reproducibility

    Returns:
        Tuple of (X_train, X_test, y_train, y_test)
    """
    print(f"Loading data from: {filepath}")

    # Load CSV file
    data = pd.read_csv(/content/concrete_data.csv)
    print(f"  Dataset shape: {data.shape}")

    # Separate features (first 8 columns) and target (last column)
    X = data.iloc[:, :-1].values  # All columns except last
    y = data.iloc[:, -1].values   # Last column (compressive strength)

    print(f"  Features shape: {X.shape}")
    print(f"  Target shape: {y.shape}")

    # Normalize features (important for neural networks)
    # Standardization: (x - mean) / std
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    X = (X - X_mean) / (X_std + 1e-8)  # Add small value to prevent division by zero

    print(f"  Features normalized (mean=0, std=1)")

    # Shuffle data for random train/test split
    np.random.seed(random_seed)
    indices = np.random.permutation(len(X))
    X = X[indices]
    y = y[indices]

    # Split into training and testing sets
    split_idx = int(len(X) * train_ratio)
    X_train = X[:split_idx]
    X_test = X[split_idx:]
    y_train = y[:split_idx]
    y_test = y[split_idx:]

    print(f"  Training samples: {len(X_train)}")
    print(f"  Testing samples: {len(X_test)}")

    return X_train, X_test, y_train, y_test

def evaluate_mae(ann: ANN, X: np.ndarray, y: np.ndarray) -> float:
    """
    Calculate Mean Absolute Error of ANN predictions

    MAE = (1/n) * Σ|predicted - actual|

    Args:
        ann: Trained ANN
        X: Input features (n_samples, n_features)
        y: True output values (n_samples,)

    Returns:
        Mean Absolute Error (lower is better)
    """
    predictions = ann.predict(X)
    mae = np.mean(np.abs(predictions.flatten() - y.flatten()))
    return mae

# ============================================================================
# MAIN EXECUTION FUNCTION
# ============================================================================

def run_single_experiment(X_train, X_test, y_train, y_test,
                         layer_sizes, activation_functions,
                         swarm_size, max_iterations, bounds,
                         num_informants=3, phi1=2.05, phi2=2.05, phi3=0.729,
                         random_seed=None):
    """
    Run a single PSO-ANN experiment

    Args:
        Data parameters:
            X_train, X_test, y_train, y_test: Dataset splits
        ANN parameters:
            layer_sizes: List of neurons per layer
            activation_functions: List of activation function names
        PSO parameters:
            swarm_size: Number of particles
            max_iterations: Number of PSO iterations
            bounds: (min, max) for parameters
            num_informants: Number of informants per particle
            phi1, phi2, phi3: PSO coefficients
        random_seed: For reproducibility

    Returns:
        Dictionary with results
    """
    if random_seed is not None:
        np.random.seed(random_seed)
        random.seed(random_seed)

    # Create ANN
    ann = ANN(layer_sizes, activation_functions)
    num_parameters = ann.get_parameter_count()

    print(f"\nANN Architecture: {layer_sizes}")
    print(f"Activation Functions: {activation_functions}")
    print(f"Total Parameters: {num_parameters}")

    # Create fitness function
    fitness_function = create_fitness_function(ann, X_train, y_train)

    # Create and run PSO
    print(f"\nPSO Configuration:")
    print(f"  Swarm Size: {swarm_size}")
    print(f"  Iterations: {max_iterations}")
    print(f"  Total Evaluations: {swarm_size * max_iterations}")
    print(f"  Bounds: {bounds}")
    print(f"  Coefficients: phi1={phi1}, phi2={phi2}, phi3={phi3}")

    pso = PSO(
        swarm_size=swarm_size,
        dimension=num_parameters,
        bounds=bounds,
        num_informants=num_informants,
        phi1=phi1,
        phi2=phi2,
        phi3=phi3
    )

    print("\nStarting PSO optimization...")
    best_parameters, best_fitness = pso.optimize(fitness_function, max_iterations)

    # Set best parameters and evaluate
    ann.set_parameters(best_parameters)
    train_mae = evaluate_mae(ann, X_train, y_train)
    test_mae = evaluate_mae(ann, X_test, y_test)

    results = {
        'train_mae': train_mae,
        'test_mae': test_mae,
        'best_fitness': best_fitness,
        'convergence_history': pso.fitness_history,
        'num_parameters': num_parameters
    }

    return results, ann, pso

def main():
    """
    Main function - demonstrates basic usage
    """
    print("="*80)
    print(" PSO-Optimized ANN for Concrete Compressive Strength Prediction")
    print("="*80)

    # Set random seed for reproducibility
    np.random.seed(42)
    random.seed(42)

    # Load and prepare data
    print("\n1. Loading and preparing data...")
    X_train, X_test, y_train, y_test = load_and_split_data('concrete_data.csv')

    # Configure ANN architecture
    print("\n2. Configuring ANN architecture...")
    layer_sizes = [8, 10, 5, 1]  # 8 inputs, 2 hidden layers, 1 output
    activation_functions = ['relu', 'relu', 'linear']

    # Configure PSO parameters
    print("\n3. Configuring PSO parameters...")
    swarm_size = 30
    max_iterations = 50
    bounds = (-5.0, 5.0)

    # Run experiment
    print("\n4. Running PSO-ANN optimization...")
    results, ann, pso = run_single_experiment(
        X_train, X_test, y_train, y_test,
        layer_sizes, activation_functions,
        swarm_size, max_iterations, bounds,
        random_seed=42
    )

    # Display results
    print("\n" + "="*80)
    print(" RESULTS")
    print("="*80)
    print(f"Training MAE:   {results['train_mae']:.4f} MPa")
    print(f"Test MAE:       {results['test_mae']:.4f} MPa")
    print(f"Best Fitness:   {results['best_fitness']:.4f}")
    print("="*80)

    # Plot convergence
    plt.figure(figsize=(10, 6))
    plt.plot(results['convergence_history'], linewidth=2)
    plt.xlabel('Iteration', fontsize=12)
    plt.ylabel('Best Fitness (MAE)', fontsize=12)
    plt.title('PSO Convergence Curve', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('pso_convergence.png', dpi=300)
    print("\nConvergence plot saved as 'pso_convergence.png'")

    return results, ann, pso

if __name__ == "__main__":
    main()

SyntaxError: invalid syntax (ipython-input-3383845189.py, line 63)